In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt


In [ ]:

import os


tracker_kcf = cv2.TrackerKCF_create()
tracker_csrt = cv2.TrackerCSRT_create()


video = cv2.VideoCapture("raccoon.mp4")


if not video.isOpened():
    print("Error: Could not open video.")
    exit()


ret, frame = video.read()
if not ret:
    print("Error: Could not read the video.")
    video.release()
    exit()

os.makedirs("output_kcf", exist_ok=True)
os.makedirs("output_csrt", exist_ok=True)


bbox = cv2.selectROI("Select Object", frame, False)
cv2.destroyWindow("Select Object")

tracker_kcf.init(frame, bbox)
tracker_csrt.init(frame, bbox)


def run_tracker(tracker, video, output_folder, tracker_name):
   
    video.set(cv2.CAP_PROP_POS_FRAMES, 0)
    frame_count = 10

    while frame_count < 45:
        ret, frame = video.read()
        if not ret:
            break

        
        success, bbox = tracker.update(frame)

        
        if success:
            p1 = (int(bbox[0]), int(bbox[1]))
            p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
            cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
        else:
            cv2.putText(frame, "Tracking failed", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)

        
        cv2.imwrite(f"{output_folder}/frame_{frame_count}.jpg", frame)

        frame_count += 1
        cv2.imshow(tracker_name, frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyWindow(tracker_name)


run_tracker(tracker_kcf, video, "output_kcf", "KCF Tracker")


run_tracker(tracker_csrt, video, "output_csrt", "CSRT Tracker")

video.release()
cv2.destroyAllWindows()
